In [1]:
import pandas as pd
import numpy as np
import importlib
import utilities.utility as utility
from pypfopt import EfficientFrontier, risk_models, expected_returns

import utilities.variables as variables
import utilities.api_ticker_service as data_ticker_service
importlib.reload(data_ticker_service)
importlib.reload(variables)

<module 'utilities.variables' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/variables.py'>

### Average historical return

In [2]:
df_monthly_adj_close = pd.read_csv('../data/10_monthly_adjacent_close.csv', index_col=0)

#### Converting 'Adj Close' Values to Percentage Change
The "Adj Close" will be transformed to represent the percentage change from the previous day's close. The percentage change will be calculated as 1 + percentage change, which indicates the relative change in the adjusted close values day-over-day.

In [3]:
df_monthly_return = df_monthly_adj_close.copy(deep=True)
for column in df_monthly_return:
    if column != 'Date':
        df_monthly_return[column] = 1 + df_monthly_adj_close[column].pct_change().round(2)
        df_monthly_return[column].dropna()

In [4]:
df_monthly_return.to_csv('../data/10_monthly_returns_complete.csv')

### Return rate

In [5]:
df_monthly_return = pd.read_csv('../data/10_monthly_returns_complete.csv', index_col=0)
df_overview = pd.read_csv('../data/data_5_scaled.csv', index_col=0)

In [6]:
df_monthly_adj_close.head()

,KE,SCS,HNI,AVT,ACCO,KEYS,CBRE,BRC,PGRE,BHE,...,ENPH,RUN,SUP,MATX,KIDS,HALO,MATW,KVHI,NEO,UNP
Date,,,,,,,,,,,,,,,,,,,,,
1999-08-01,NaN,6.013282,11.762950,17.141674,NaN,NaN,NaN,8.704611,NaN,13.926089,...,NaN,NaN,15.431546,6.066541,NaN,NaN,9.999043,2.43750,NaN,7.509912
1999-09-01,NaN,5.658068,9.648345,16.270067,NaN,NaN,NaN,9.284917,NaN,13.358641,...,NaN,NaN,15.431546,6.040688,NaN,NaN,11.104928,2.37500,NaN,7.413507
1999-10-01,NaN,5.165832,9.836304,21.209169,NaN,NaN,NaN,8.414455,NaN,6.052765,...,NaN,NaN,14.708194,6.072314,NaN,NaN,9.261786,3.06250,NaN,8.634527
1999-11-01,NaN,5.319273,10.838723,21.354946,NaN,NaN,NaN,8.951837,NaN,8.417130,...,NaN,NaN,14.823595,5.756048,NaN,NaN,8.586853,3.03125,NaN,7.289008
1999-12-01,NaN,4.910099,11.044152,23.517157,NaN,NaN,NaN,9.899887,NaN,8.677205,...,NaN,NaN,14.823595,5.827809,NaN,NaN,10.156491,3.06250,NaN,6.766294


In [7]:
# move Date index to a new column
df_monthly_adj_close = df_monthly_adj_close.reset_index(level=0)
df_monthly_return = df_monthly_return.reset_index(level=0)

# Creates a new column for each year group (1, 5, 10, 25 year returns)
utility.set_yearly_return_rates_by_years(df_overview, df_monthly_return);

### Volatility
To calculate the volatility of a stock's monthly returns, there are several steps that should be applied to the historical price data.
* Calculate Monthly Returns
* Calculate the Average Monthly Return
* Calculate the Monthly Return Deviations
* Square the Deviations
* Calculate the Variance
* Calculate the Standard Deviation

In [8]:
utility.set_volatility_by_years(df_overview, df_monthly_adj_close)

In [9]:
df_overview[['stock_ticker_symbol', 'return_rate_5y_avg', 'return_rate_10y_avg', 'return_rate_25y_avg', 'volatility_10y']].sort_values(by=['return_rate_10y_avg', 'volatility_10y'])

,stock_ticker_symbol,return_rate_5y_avg,return_rate_10y_avg,return_rate_25y_avg,volatility_10y
442,FOSL,0.625339,0.647001,0.913120,0.239848
1453,GPRO,0.787331,0.663986,0.871982,0.164852
1235,AMC,0.547253,0.702343,0.871146,0.568230
1594,BIG,0.547151,0.706566,0.893492,0.193526
897,WW,0.494756,0.727525,0.877270,0.281869
...,...,...,...,...,...
1651,9260.T,NaN,NaN,NaN,NaN
1652,DEQ.DE,NaN,NaN,NaN,NaN
1656,9842.T,NaN,NaN,NaN,NaN
1658,MOON.L,NaN,NaN,NaN,NaN


### Handling Categories

### Store changes

In [10]:
df_overview.to_csv('../data/data_10_overview.csv')
df_monthly_return.to_csv('../data/data_10_monthly_returns_complete.csv')